In [2]:
!pip install -q peft transformers bitsandbytes datasets fsspec==2025.3.0 evaluate pycocoevalcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.0 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model
import matplotlib.pyplot as plt
from collections import defaultdict
import evaluate
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import zipfile

zip_path = "/content/drive/MyDrive/UPenn 2025 fall courses/ESE 5460/proj/flickr8k.zip"
extract_path = "/content/flickr8k"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [6]:
if not os.path.exists("flickr8k_captions.csv"):
    df = pd.read_csv("flickr8k/captions.txt")
    df = df.dropna().reset_index(drop=True)
    df.to_csv("flickr8k_captions.csv", index=False)

In [7]:
import numpy as np
import pandas as pd

df_all = pd.read_csv("flickr8k_captions.csv")

if "image" in df_all.columns:
    image_col = "image"
elif "image_file" in df_all.columns:
    image_col = "image_file"
else:
    raise ValueError(f"Cannot find image column in csv. Columns are: {df_all.columns}")

all_images = df_all[image_col].unique()

np.random.seed(42)
np.random.shuffle(all_images)

train_images = all_images[:6000]
val_images   = all_images[6000:7000]
test_images  = all_images[7000:8000]

In [8]:
print("Total unique images:", len(all_images))
print("Train images:", len(train_images))
print("Val images:", len(val_images))
print("Test images:", len(test_images))


Total unique images: 8091
Train images: 6000
Val images: 1000
Test images: 1000


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoProcessor, Blip2ForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")

# fp16-sharded
model = Blip2ForConditionalGeneration.from_pretrained(
    "ybelkada/blip2-opt-2.7b-fp16-sharded",
    device_map="auto",
    load_in_8bit=True
)

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
model.config.image_token_id = image_token
model.generation_config.image_token_id = image_token
print("Fixed image_token_id:", model.config.image_token_id)

# 8-bit LoRA
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.01,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/995M [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Fixed image_token_id: 50265
trainable params: 3,932,160 || all params: 3,748,612,096 || trainable%: 0.1049


In [ ]:
class Flickr8kDataset(Dataset):
    def __init__(self, captions_file, image_dir, processor, image_list):
        self.image_dir = image_dir
        self.processor = processor

        df = pd.read_csv(captions_file)

        df = df[df["image"].isin(image_list)].reset_index(drop=True)

        self.samples = df

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        row = self.samples.iloc[idx]
        image_path = os.path.join(self.image_dir, row["image"])
        caption = row["caption"]

        image = Image.open(image_path).convert("RGB")

        inputs = self.processor(
            images=image,
            text=caption,
            padding="max_length",
            truncation=True,
            max_length=64,
            return_tensors="pt"
        )

        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs


In [ ]:
# ===== create Dataset =====
train_dataset = Flickr8kDataset(
    "flickr8k_captions.csv",
    "flickr8k/Images",
    processor,
    image_list=train_images
)

val_dataset = Flickr8kDataset(
    "flickr8k_captions.csv",
    "flickr8k/Images",
    processor,
    image_list=val_images
)

test_dataset = Flickr8kDataset(
    "flickr8k_captions.csv",
    "flickr8k/Images",
    processor,
    image_list=test_images
)

In [ ]:
# ===== key sanity check =====
print("Train caption rows:", len(train_dataset))
print("Val caption rows:", len(val_dataset))
print("Test caption rows:", len(test_dataset))

print("Train unique images:", train_dataset.samples["image"].nunique())
print("Val unique images:", val_dataset.samples["image"].nunique())
print("Test unique images:", test_dataset.samples["image"].nunique())


Train caption rows: 30000
Val caption rows: 5000
Test caption rows: 5000
Train unique images: 6000
Val unique images: 1000
Test unique images: 1000


In [ ]:
def execute_training_cycle(model, dataloader, optimizer, device, epochs=2):
    model.train()

    for current_epoch in range(epochs):
        epoch_loss = 0.0

        print(f"\nCommencing training epoch {current_epoch + 1}/{epochs}")

        for batch_idx, data_batch in enumerate(dataloader):
            input_ids = data_batch["input_ids"].to(device)
            attention_mask = data_batch["attention_mask"].to(device)
            pixel_values = data_batch["pixel_values"].to(device, torch.float16)

            # create labels
            labels = input_ids.clone()
            labels[labels == processor.tokenizer.pad_token_id] = -100

            model_inputs = {
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "pixel_values": pixel_values,
                "labels": labels,
            }

            optimizer.zero_grad()

            outputs = model(**model_inputs)
            loss = outputs.loss.float()

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            if batch_idx % 10 == 0:
                avg_loss = epoch_loss / (batch_idx + 1)
                print(
                    f"Epoch {current_epoch + 1} | Batch {batch_idx} "
                    f"| Avg Loss: {avg_loss:.4f} | Current Loss: {loss.item():.4f}"
                )

        epoch_mean_loss = epoch_loss / len(dataloader)
        print(f"\nEpoch {current_epoch + 1} completed")
        print(f"Average epoch loss: {epoch_mean_loss:.4f}")
        print("-" * 50)


In [ ]:
# ===== use train_images create train dataset =====
train_dataset = Flickr8kDataset(
    "flickr8k_captions.csv",
    "flickr8k/Images",
    processor,
    image_list=train_images
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

execute_training_cycle(
    model,
    train_dataloader,
    optimizer,
    device,
    epochs=2
)



Commencing training epoch 1/2


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Se

Epoch 1 | Batch 0 | Avg Loss: 12.8733 | Current Loss: 12.8733
Epoch 1 | Batch 10 | Avg Loss: 10.1234 | Current Loss: 6.6275
Epoch 1 | Batch 20 | Avg Loss: 7.3924 | Current Loss: 3.6874
Epoch 1 | Batch 30 | Avg Loss: 5.8514 | Current Loss: 1.7942
Epoch 1 | Batch 40 | Avg Loss: 4.7240 | Current Loss: 0.8943
Epoch 1 | Batch 50 | Avg Loss: 3.9478 | Current Loss: 0.6600
Epoch 1 | Batch 60 | Avg Loss: 3.4114 | Current Loss: 0.6782
Epoch 1 | Batch 70 | Avg Loss: 3.0263 | Current Loss: 0.5331
Epoch 1 | Batch 80 | Avg Loss: 2.7300 | Current Loss: 0.6150
Epoch 1 | Batch 90 | Avg Loss: 2.4959 | Current Loss: 0.6216
Epoch 1 | Batch 100 | Avg Loss: 2.3078 | Current Loss: 0.6503
Epoch 1 | Batch 110 | Avg Loss: 2.1536 | Current Loss: 0.5430
Epoch 1 | Batch 120 | Avg Loss: 2.0225 | Current Loss: 0.5533
Epoch 1 | Batch 130 | Avg Loss: 1.9141 | Current Loss: 0.5325
Epoch 1 | Batch 140 | Avg Loss: 1.8221 | Current Loss: 0.6425
Epoch 1 | Batch 150 | Avg Loss: 1.7381 | Current Loss: 0.5245
Epoch 1 | Batch 

##download the adapter

In [ ]:
save_dir = "/content/drive/MyDrive/UPenn 2025 fall courses/ESE 5460/proj/blip2_lora_flickr8k"
model.save_pretrained(save_dir)
processor.save_pretrained(save_dir)

['/content/drive/MyDrive/UPenn 2025 fall courses/ESE 5460/proj/blip2_lora_flickr8k/processor_config.json']

##load the model

In [ ]:
from peft import PeftModel
from transformers import Blip2ForConditionalGeneration, AutoProcessor

device = "cuda"

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")

base_model = Blip2ForConditionalGeneration.from_pretrained(
    "ybelkada/blip2-opt-2.7b-fp16-sharded",
    device_map="auto",
    load_in_8bit=True
)

model = PeftModel.from_pretrained(base_model, save_dir)
model.eval()

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
model.config.image_token_id = image_token
model.generation_config.image_token_id = image_token

model.print_trainable_parameters()



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

trainable params: 0 || all params: 3,748,612,096 || trainable%: 0.0000


In [ ]:
model.eval()

def generate_caption(image_path):
    """Generate a caption for a single image"""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

    with torch.no_grad():
        generated_ids = model.generate(
            pixel_values=inputs.pixel_values,
            max_new_tokens=50,
            num_beams=3,
            repetition_penalty=1.2
        )

    return processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )[0].strip()

def plot_images_with_captions(image_paths, captions):
    plt.figure(figsize=(18, 10))
    for i, (img_path, caption) in enumerate(zip(image_paths, captions)):
        img = Image.open(img_path).convert("RGB")
        plt.subplot(2, 3, i + 1)
        plt.imshow(img)
        plt.title(caption, wrap=True, fontsize=11)
        plt.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
image_dir = "flickr8k/Images"

image_files = [
    os.path.join(image_dir, img)
    for img in val_images[:5]
]

# Generate captions
captions = [generate_caption(img_path) for img_path in image_files]

# Display results
print("Generated Captions (Validation Set):")
for img_path, caption in zip(image_files, captions):
    print(f"{os.path.basename(img_path)}: {caption}")

plot_images_with_captions(image_files, captions)


Output hidden; open in https://colab.research.google.com to view.

### Generate caption using test set

In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image

# ===============================
# Save directory (Drive)
# ===============================
SAVE_ROOT = "/content/drive/MyDrive/UPenn 2025 fall courses/ESE 5460/proj"
EXP_NAME = "blip2_lora_flickr8k_test_preds"
os.makedirs(SAVE_ROOT, exist_ok=True)

pred_save_path = os.path.join(SAVE_ROOT, f"{EXP_NAME}.csv")

# ===============================
# Generate captions
# ===============================
model.eval()

rows = []

for img in tqdm(test_images, desc="Generating captions"):
    image_path = os.path.join("/content/flickr8k/Images", img)
    image = Image.open(image_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

    with torch.no_grad():
        generated_ids = model.generate(
            pixel_values=inputs.pixel_values,
            max_new_tokens=25,
            num_beams=3,
            repetition_penalty=1.2
        )

    pred_caption = processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )[0].strip()

    rows.append({
        "image_file": img,
        "prediction": pred_caption
    })

# ===============================
# Save to CSV
# ===============================
pred_df = pd.DataFrame(rows)
pred_df.to_csv(pred_save_path, index=False)

print(f"Predictions saved to: {pred_save_path}")
print(f"Total images: {len(pred_df)}")


Generating captions:   1%|          | 12/1000 [00:07<10:15,  1.60it/s]


KeyboardInterrupt: 

### Image-level Evaluation with BLEU and CIDEr

We first evaluate the fine-tuned BLIP-2 model using standard image captioning metrics,
namely BLEU and CIDEr. Evaluation is conducted at the image level: for each image, the
model generates a single caption, which is then compared against all available ground-truth
captions for that image.

BLEU measures n-gram overlap between the generated caption and reference captions,
while CIDEr is specifically designed for image captioning and assigns higher weights
to n-grams that are more informative and discriminative across the dataset.


In [ ]:
import os
from collections import defaultdict
import pandas as pd
import evaluate

from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer

# ===============================
# Load predictions
# ===============================
SAVE_ROOT = "/content/drive/MyDrive/UPenn 2025 fall courses/ESE 5460/proj"
EXP_NAME = "blip2_lora_flickr8k_test_preds"

pred_save_path = os.path.join(SAVE_ROOT, f"{EXP_NAME}.csv")
pred_df = pd.read_csv(pred_save_path)

print(f"Loaded predictions: {len(pred_df)}")

# ===============================
# Load ground-truth captions
# ===============================
caption_df = pd.read_csv(
    "/content/flickr8k/captions.txt",
    names=["image_file", "caption_text"]
)

image_to_captions = defaultdict(list)
for _, row in caption_df.iterrows():
    image_to_captions[row["image_file"]].append(row["caption_text"])

# ===============================
# Prepare evaluation lists
# ===============================
predictions = []
references = []

for _, row in pred_df.iterrows():
    img = row["image_file"]
    predictions.append(row["prediction"])
    references.append(image_to_captions[img])

# ===============================
# BLEU
# ===============================
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(
    predictions=predictions,
    references=references
)

print(f"BLEU score: {bleu_score['bleu']:.4f}")

# ===============================
# CIDEr
# ===============================
cider = Cider()
tokenizer = PTBTokenizer()

gt_dict = {}
res_dict = {}

for i, row in pred_df.iterrows():
    img = row["image_file"]
    gt_dict[str(i)] = [{"caption": c} for c in image_to_captions[img]]
    res_dict[str(i)] = [{"caption": row["prediction"]}]

gt_tok = tokenizer.tokenize(gt_dict)
res_tok = tokenizer.tokenize(res_dict)

cider_score, _ = cider.compute_score(gt_tok, res_tok)
print(f"CIDEr score: {cider_score:.4f}")


Loaded predictions: 1000


BLEU score: 0.1670
CIDEr score: 0.9722


### Semantic and Cross-Modal Evaluation with BERTScore and CLIPScore

In addition to traditional n-gram based metrics, we further report BERTScore and CLIPScore
to better capture semantic similarity and image–text alignment.
BERTScore evaluates the semantic similarity between generated captions and reference captions
using contextualized token embeddings, which is more robust to paraphrasing.
CLIPScore measures the alignment between an image and its generated caption in a shared
vision–language embedding space, directly reflecting cross-modal consistency.


In [ ]:
# BERTScore dependency
!pip install bert_score

# CLIPScore dependencies (official OpenAI CLIP)
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rppt2e34
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rppt2e34
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=7d586a6c07850ce1684cd7a94a372f032bf75fd326fdc9689b853e2fd5a9a9e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-r68epp3a/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
import os
import numpy as np
import torch
import evaluate
import clip
from PIL import Image
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# ===============================
# Prepare predictions & references
# ===============================
predictions = []
flat_references = []
image_paths = []

for _, row in pred_df.iterrows():
    img = row["image_file"]
    predictions.append(row["prediction"])

    # BERTScore: one reference per image
    flat_references.append(image_to_captions[img][0])

    # CLIPScore: image path
    image_paths.append(
        os.path.join("/content/flickr8k/Images", img)
    )

# ===============================
# BERTScore (semantic similarity)
# ===============================
bertscore = evaluate.load("bertscore")

bert_results = bertscore.compute(
    predictions=predictions,
    references=flat_references,
    lang="en",
    model_type="roberta-large"
)

bert_f1 = float(np.mean(bert_results["f1"]))
print(f"BERTScore (F1): {bert_f1:.4f}")

# ===============================
# CLIPScore (image-text alignment)
# ===============================
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

clip_scores = []

for img_path, caption in tqdm(
    zip(image_paths, predictions),
    total=len(predictions),
    desc="Computing CLIPScore"
):
    image = clip_preprocess(
        Image.open(img_path).convert("RGB")
    ).unsqueeze(0).to(device)

    text = clip.tokenize([caption]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        text_features = clip_model.encode_text(text)

        # Normalize embeddings
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # Cosine similarity
        score = (image_features * text_features).sum().item()
        clip_scores.append(score)

clip_score = float(np.mean(clip_scores))
print(f"CLIPScore: {clip_score:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.8694


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 129MiB/s]
Computing CLIPScore: 100%|██████████| 1000/1000 [00:26<00:00, 37.85it/s]

CLIPScore: 0.3075


In [ ]:
# =========================================
# Rebuild image_to_captions (GT captions)
# =========================================

import pandas as pd
from collections import defaultdict

caption_df = pd.read_csv(
    "/content/flickr8k/captions.txt",
    names=["image_file", "caption"]
)

image_to_captions = defaultdict(list)
for _, row in caption_df.iterrows():
    image_to_captions[row["image_file"]].append(row["caption"])

print("image_to_captions rebuilt.")
print("Example:", list(image_to_captions.items())[:1])


image_to_captions rebuilt.
Example: [('image', ['caption'])]


In [ ]:
# =========================================
# Qualitative Analysis Export (VAL SET)
# SMALL CANVAS = BIG TEXT (NO WHITESPACE)
# =========================================

import os
import textwrap
from PIL import Image, ImageDraw, ImageFont

# ===============================
# CONFIG
# ===============================
SAVE_ROOT = "/content/drive/MyDrive/UPenn 2025 fall courses/ESE 5460/proj/eva_analysis"

IMAGE_DIR = "/content/flickr8k/Images"

CANVAS_W = 400
CANVAS_H = 140

LEFT_W = 200
RIGHT_W = 200

FONT_SIZE = 50
LINE_SPACING = 10


# ===============================
# FONT
# ===============================
def load_font(size):
    try:
        return ImageFont.truetype(
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            size
        )
    except:
        return ImageFont.load_default()

font = load_font(FONT_SIZE)


# ===============================
# TEXT DRAWING
# ===============================
def draw_wrapped_text(draw, text, x, y, max_chars):
    lines = textwrap.wrap(text, max_chars)
    cur_y = y
    for line in lines:
        draw.text((x, cur_y), line, fill=(0, 0, 0), font=font)
        cur_y += font.size + LINE_SPACING


# ===============================
# CREATE PNG (NO WHITESPACE)
# ===============================
def create_png(image, caption):
    canvas = Image.new("RGB", (CANVAS_W, CANVAS_H), (255, 255, 255))
    draw = ImageDraw.Draw(canvas)

    # ---- Image fills LEFT half ----
    image = image.resize((LEFT_W, CANVAS_H))
    canvas.paste(image, (0, 0))

    # ---- Caption fills RIGHT half ----
    draw_wrapped_text(
        draw,
        caption,
        x=LEFT_W + 12,
        y=20,
        max_chars=30
    )

    return canvas


# ===============================
# MAIN EXPORT
# ===============================
def export_val_qualitative_samples(num_samples=1):
    os.makedirs(SAVE_ROOT, exist_ok=True)

    for i in range(num_samples):
        img_name = val_images[i]
        sample_dir = os.path.join(SAVE_ROOT, f"sample_{i:02d}")
        os.makedirs(sample_dir, exist_ok=True)

        image_path = os.path.join(IMAGE_DIR, img_name)
        image = Image.open(image_path).convert("RGB")

        gt_caption = image_to_captions[img_name][0]
        pred_caption = generate_caption(image_path)

        gt_png = create_png(image, gt_caption)
        pred_png = create_png(image, pred_caption)

        gt_png.save(os.path.join(sample_dir, "gt.png"))
        pred_png.save(os.path.join(sample_dir, "pred.png"))

        print(f"✔ sample_{i:02d} saved")

    print("✅ Done.")


In [ ]:
export_val_qualitative_samples(num_samples=30)


✔ sample_00 saved
✔ sample_01 saved
✔ sample_02 saved
✔ sample_03 saved
✔ sample_04 saved
✔ sample_05 saved
✔ sample_06 saved
✔ sample_07 saved
✔ sample_08 saved
✔ sample_09 saved
✔ sample_10 saved
✔ sample_11 saved
✔ sample_12 saved
✔ sample_13 saved
✔ sample_14 saved
✔ sample_15 saved
✔ sample_16 saved
✔ sample_17 saved
✔ sample_18 saved
✔ sample_19 saved
✔ sample_20 saved
✔ sample_21 saved
✔ sample_22 saved
✔ sample_23 saved
✔ sample_24 saved
✔ sample_25 saved
✔ sample_26 saved
✔ sample_27 saved
✔ sample_28 saved
✔ sample_29 saved
✅ Done.
